<center> 
    <h1> Speech Transformer : A Speech to Text Transformer in TensorFlow 2 </h1>
    <h2> Librispeech Preprocessing </h2>
</center>

In [ ]:
!pip install tensorflow-io

In [ ]:
import librosa
import pickle
import os
import sys
import tensorflow as tf
import tensorflow_io as tfio

#Librispeech Download

In [ ]:
#6.3GB 100 hours training dataset
!curl --header "Host: www.openslr.org" --header "User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36" --header "Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header "Accept-Language: en-GB,en-US;q=0.9,en;q=0.8" "http://www.openslr.org/resources/12/train-clean-100.tar.gz" -L -o 'train-clean-100.tar.gz'
!tar xvzf train-clean-100.tar.gz

#23GB 360 hours training dataset
!curl --header "Host: www.openslr.org" --header "User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36" --header "Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header "Accept-Language: en-GB,en-US;q=0.9,en;q=0.8" --header "Referer: http://www.openslr.org/12" "http://www.openslr.org/resources/12/train-clean-360.tar.gz" -L -o 'train-clean-360.tar.gz'
!tar xvzf train-clean-360.tar.gz

#30GB 500 hours training dataset
!curl --header "Host: www.openslr.org" --header "User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36" --header "Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header "Accept-Language: en-GB,en-US;q=0.9,en;q=0.8" --header "Referer: http://www.openslr.org/12" "http://www.openslr.org/resources/12/train-other-500.tar.gz" -L -o 'train-other-500.tar.gz'
!tar xvzf train-other-500.tar.gz

#dev-clean validation set
!curl --header "Host: www.openslr.org" --header "User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36" --header "Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header "Accept-Language: en-GB,en-US;q=0.9,en;q=0.8" --header "Referer: http://www.openslr.org/12/" "http://www.openslr.org/resources/12/dev-clean.tar.gz" -L -o 'dev-clean.tar.gz'
!tar xvzf dev-clean.tar.gz

#Training set preprocessing

##Train Clean

In [ ]:
spec = []
speech = []

In [ ]:
path = 'LibriSpeech/train-clean-100'
i=1
for rep1 in os.listdir(path):
    sys.stdout.write("\r{}/{}".format(i,len(os.listdir(path))))
    i+=1
    for rep2 in os.listdir(os.path.join(path,rep1)):
        for file in os.listdir(os.path.join(path,rep1,rep2)):
            if file[-4:] == '.txt':
                file = open(os.path.join(path,rep1,rep2,file),'r')
                for line in file.readlines():
                    A = librosa.core.load(os.path.join(path,rep1,rep2,line.split()[0] + '.flac'),sr=16000)[0]
                    
                    S = tfio.experimental.audio.spectrogram(A, nfft=512, window=400, stride=160)
                    S = tfio.experimental.audio.melscale(S, rate=16000, mels=80, fmin=0, fmax=8000)
                    S = tf.math.log(S+1e-9)
                    S = (S - tf.math.reduce_mean(S))/tf.math.reduce_std(S)
                    S = tf.cast(25*S, tf.int8)
                    
                    spec.append(S)
                    speech.append(line[len(line.split()[0]):].lower().replace('\n','')[1:])
                    

In [ ]:
print("{} audio".format(len(spec)))
print("{} speech".format(len(speech)))

In [ ]:
with open("pickled_LibriSpeech_clean100_10ms.pkl", "bw") as f:
    data = (spec, speech)
    pickle.dump(data, f, protocol=4)

In [ ]:
path = 'LibriSpeech/train-clean-360'
i=1
for rep1 in os.listdir(path):
    sys.stdout.write("\r{}/{}".format(i,len(os.listdir(path))))
    i+=1
    for rep2 in os.listdir(os.path.join(path,rep1)):
        for file in os.listdir(os.path.join(path,rep1,rep2)):
            if file[-4:] == '.txt':
                file = open(os.path.join(path,rep1,rep2,file),'r')
                for line in file.readlines():
                    A = librosa.core.load(os.path.join(path,rep1,rep2,line.split()[0] + '.flac'),sr=16000)[0]
                    
                    S = tfio.experimental.audio.spectrogram(A, nfft=512, window=400, stride=160)
                    S = tfio.experimental.audio.melscale(S, rate=16000, mels=80, fmin=0, fmax=8000)
                    S = tf.math.log(S+1e-9)
                    S = (S - tf.math.reduce_mean(S))/tf.math.reduce_std(S)
                    S = tf.cast(25*S, tf.int8)
                    
                    spec.append(S)
                    speech.append(line[len(line.split()[0]):].lower().replace('\n','')[1:])
                    
      

In [ ]:
print("{} audio".format(len(spec)))
print("{} speech".format(len(speech)))

In [ ]:
with open("pickled_LibriSpeech_clean_10ms.pkl", "bw") as f: # train-clean-100 + train-clean-360
    data = (spec, speech)
    pickle.dump(data, f, protocol=4)

##Train Other

In [ ]:
spec = []
speech = []

In [ ]:
path = 'LibriSpeech/train-other-500'
i=1
for rep1 in os.listdir(path):
    sys.stdout.write("\r{}/{}".format(i,len(os.listdir(path))))
    i+=1
    for rep2 in os.listdir(os.path.join(path,rep1)):
        for file in os.listdir(os.path.join(path,rep1,rep2)):
            if file[-4:] == '.txt':
                file = open(os.path.join(path,rep1,rep2,file),'r')
                for line in file.readlines():
                    A = librosa.core.load(os.path.join(path,rep1,rep2,line.split()[0] + '.flac'),sr=16000)[0]
                    
                    S = tfio.experimental.audio.spectrogram(A, nfft=512, window=400, stride=160)
                    S = tfio.experimental.audio.melscale(S, rate=16000, mels=80, fmin=0, fmax=8000)
                    S = tf.math.log(S+1e-9)
                    S = (S - tf.math.reduce_mean(S))/tf.math.reduce_std(S)
                    S = tf.cast(25*S, tf.int8)
                    
                    spec.append(S)
                    speech.append(line[len(line.split()[0]):].lower().replace('\n','')[1:])
                    
         

In [ ]:
print("{} audio".format(len(spec)))
print("{} speech".format(len(speech)))

In [ ]:
with open("pickled_LibriSpeech_other_10ms.pkl", "bw") as f:
    data = (spec, speech)
    pickle.dump(data, f, protocol=4)

#Validation set preprocessing

##Dev Clean

In [ ]:
spec = []
speech = []

In [ ]:
path = 'LibriSpeech/dev-clean'
i=1
for rep1 in os.listdir(path):
    sys.stdout.write("\r{}/{}".format(i,len(os.listdir(path))))
    i+=1
    for rep2 in os.listdir(os.path.join(path,rep1)):
        for file in os.listdir(os.path.join(path,rep1,rep2)):
            if file[-4:] == '.txt':
                file = open(os.path.join(path,rep1,rep2,file),'r')
                for line in file.readlines():
                    A = librosa.core.load(os.path.join(path,rep1,rep2,line.split()[0] + '.flac'),sr=16000)[0]
                    
                    S = tfio.experimental.audio.spectrogram(A, nfft=512, window=400, stride=160)
                    S = tfio.experimental.audio.melscale(S, rate=16000, mels=80, fmin=0, fmax=8000)
                    S = tf.math.log(S+1e-9)
                    S = (S - tf.math.reduce_mean(S))/tf.math.reduce_std(S)
                    S = tf.cast(25*S, tf.int8)
                    
                    spec.append(S)
                    speech.append(line[len(line.split()[0]):].lower().replace('\n','')[1:])
                    
        

In [ ]:
print("{} audio".format(len(spec)))
print("{} speech".format(len(speech)))

In [ ]:
with open("pickled_LibriSpeech_dev_clean_10ms.pkl", "bw") as f:
    data = (spec, speech)
    pickle.dump(data, f, protocol=4)